In [ ]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

from gen_ai_hub.proxy.native.openai import chat
from gen_ai_hub.proxy.native.google_vertexai.clients import GenerativeModel
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client

In [ ]:
import os
import tempfile
import git

IRRELEVANT_FILES = [
    "webapp/",
    "app/",
    "node_modules/",
    ".gitignore",
    "package-lock.json",
    "i18n/",
    "i18n"
    ".vscode",
    ".env",
    "eslint",
    ".eslintrc",
    ".git/",
    "__pycache__/",
    "core"
]
    
def collect_all_files(repo_url, irrelevant_files=IRRELEVANT_FILES):
    
    project_path = tempfile.mkdtemp()
    git.Repo.clone_from(repo_url, project_path)
    print(f"Cloned repository to {project_path}")
    
    relevant_files = []
    n_total=0
    n_relevant=0
    
    for root, dirs, files in os.walk(project_path):
        for file in files:
            
            abs_path = os.path.join(root, file)
            rel_path = os.path.relpath(abs_path, project_path)
                            
            n_total += 1
            
            if any(exclude in rel_path or exclude in files for exclude in irrelevant_files):
                n_relevant += 1

            relevant_files.append({
                "name": file,
                "full_path": abs_path,
                "rel_path": rel_path,
                "format": file.split(".")[-1] if "." in file else None,
                "is_relevant": not any(exclude in rel_path or exclude in files for exclude in irrelevant_files),    
            })

    print(f"Total files: {n_total}")
    print(f"Relevant files: {n_relevant}")
    
    return relevant_files



In [ ]:
repo_files = collect_all_files("https://github.tools.sap/Delivery-Scale-PT/opportunity-assistant-CAP.git")
print(f"Collected {len(repo_files)} files from the repository.")

In [ ]:
def generate_cds_documentation(relevant_file_content, llm_model = "gpt-4o", temperature = 0):
    
    system_prompt = """
        You are a technical documentation assistant specialized in SAP CAP (Cloud Application Programming Model). 
        You will receive the full content of a CAP project, including all relevant .cds files, service implementations (.js), and metadata/configuration files.

        Your task is to generate complete, structured, and developer-friendly technical documentation in **Markdown** format. The documentation must be organized into the following clearly labeled sections:

        ---

        1. **Project File Structure**  
        - Provide a full hierarchical folder and file structure of the CAP project using indentation or tree-style formatting.

        2. **CAP Application Files Overview**  
        - Present a two-column table:  
            | File Name | Description |
        - Describe the purpose and role of each file in the application context.

        3. **Data Model Representation**  
        - Use a Markdown diagram or text-based representation showing:
            - All entities
            - Attributes (name, type, default, key)
            - Associations and compositions with cardinality (1:1, 1:n, etc.)
            - Navigation paths between entities

        4. **DB Schema**  
        - Cover all Tables, Views, and Types.
        - Use a table with the following columns:  
            | Name | Type (Table/View/Type) | Fields (name, key, type, default, annotations) | Annotations | Description |

        5. **CDS Definitions**  
        - For each CDS definition, present a table including:
            | Name | Projected DB Entity | Supported CRUD Operations | Fields (name, type, annotations) | Description | Annotations |

        6. **Function and Action Imports**  
        - Provide a table with the following columns:  
            | Name | Type (Function/Action) | Description | Supported Operation (GET/POST) | Parameters (name, type, default) | Return Type | Annotations | Associated Entities |

        7. **Event Handlers**  
        - For every handler in the implementation layer, document:  
            | Handler Type (on/before/after) | Operation (Create/Read/Update/Delete/Custom) | Target (Entity/Function/Action) | Implementation Description | Helper Functions Used |

        8. **Server Helper Functions**  
        - Extract and describe reusable helper functions defined in the `srv/` folder:  
            | Function Name | Location (relative file path) | Description | Parameters (name, type, default) | Return Type | Implementation Logic Summary |

        ---

        **⚠️ Output Instructions – Must Follow Without Exception**:
        - **Do not** omit or leave any section blank.
        - For **every detail requested**, include a placeholder if the value is missing:
            - `[N/A]` — Not applicable for this context.
            - `[EMPTY]` — Expected but no value was found in the input.
            - `[UNKNOWN]` — Could not be deduced or inferred confidently.
        - **Never** leave any field or table cell empty.
        - Avoid raw code unless absolutely necessary.
        - Provide clear, concise, and technical descriptions suitable for SAP CAP developers.

        The documentation must be copy-paste ready for Markdown viewers or Microsoft Word.
    """

    user_prompt = f"""
                    Here are the CAP project files and their contents:

                    {relevant_file_content}
                    """
                    
    # proxy_client = get_proxy_client('gen-ai-hub')
    # kwargs = dict({'model_name': llm_model})
    # model = GenerativeModel(proxy_client=proxy_client, **kwargs)

    # content = [{
    #     "role": "user",
    #     "parts": [{
    #         "text": system_prompt + user_prompt,
    #     }]
    # }]
    
    # return model.generate_content(content).candidates[0].content.parts[0].text

    response = chat.completions.create(
                    model=llm_model,
                    temperature=temperature,
                    messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": user_prompt}
                    ]
                )
    
    return response.choices[0].message.content

In [ ]:
import os

def load_file_content(file_list, only_relevant=True):
    content = ""

    for file in file_list: 
        header = (
                    f"\n\n===== FILE: {file['name']} =====\n"
                    f"Path   : {file['rel_path']}\n"
                    f"Format : {file['format']}\n"
                )
        
        if only_relevant and not file["is_relevant"]:
            file_content = "THIS FILE'S CONTENT IS IRRELEVANT FOR CAP PROJECT DOCUMENTATION"
        else:
            try:
                with open(file["full_path"], "r", encoding="utf-8") as f:
                    file_content = f.read()
            except Exception as e:
                file_content = f"⚠️ Skipping file {file.get('rel_path', 'unknown')} due to error: {e}"

        content += f"{header}\n{file_content}\n"
    
    return content

In [ ]:
result = generate_cds_documentation(
            relevant_file_content = load_file_content(
                repo_files, only_relevant=True
            ),
            llm_model = "gpt-4o",
            temperature = 0.3
        )

In [ ]:
print(result)

# SAP CAP Project Documentation

## 1. Project File Structure
```
.
├── .git
│   ├── config
│   ├── description
│   ├── HEAD
│   ├── hooks
│   │   ├── applypatch-msg.sample
│   │   ├── commit-msg.sample
│   │   ├── fsmonitor-watchman.sample
│   │   ├── post-update.sample
│   │   ├── pre-applypatch.sample
│   │   ├── pre-commit.sample
│   │   ├── prepare-commit-msg.sample
│   │   ├── pre-merge-commit.sample
│   │   ├── pre-rebase.sample
│   │   ├── pre-receive.sample
│   │   ├── pre-push.sample
│   │   ├── push-to-checkout.sample
│   │   ├── update.sample
│   ├── index
│   ├── info
│   │   └── exclude
│   ├── logs
│   │   ├── HEAD
│   │   └── refs
│   │       ├── heads
│   │       │   └── main
│   │       └── remotes
│   │           └── origin
│   │               └── HEAD
│   ├── objects
│   │   └── pack
│   │       ├── pack-4ee560a8274540e3aa55a3ee64ccc74383bf8635.idx
│   │       └── pack-4ee560a8274540e3aa55a3ee64ccc74383bf8635.pack
│   ├── packed-refs
│   └── refs
│       ├── heads
│       │   └── main
│       └── remotes
│           └── origin
│               └── HEAD
├── .vscode
│   ├── extensions.json
│   ├── launch.json
│   └── tasks.json
├── app
│   ├── opportunityassistantcockpit
│   │   ├── controller
│   │   │   ├── App.controller.js
│   │   │   ├── Assistant.controller.js
│   │   │   ├── BaseController.js
│   │   │   ├── Files.controller.js
│   │   │   ├── Home.controller.js
│   │   │   ├── Settings.controller.js
│   │   │   └── Sources.controller.js
│   │   ├── css
│   │   │   └── style.css
│   │   ├── i18n
│   │   │   ├── i18n.properties
│   │   │   ├── i18n_en.properties
│   │   │   ├── terminologies
│   │   │   │   ├── sports
│   │   │   │   │   ├── i18n.terminologies.sports_de.properties
│   │   │   │   │   └── i18n.terminologies.sports_en.properties
│   │   │   │   └── travel
│   │   │   │       ├── i18n.terminologies.travel_de.properties
│   │   │   │       └── i18n.terminologies.travel_en.properties
│   │   ├── lib
│   │   │   ├── axios.min.js
│   │   │   ├── opencv.js
│   │   │   └── tfjs.js
│   │   ├── model
│   │   │   ├── formatter.js
│   │   │   ├── models.js
│   │   │   └── sideContent.json
│   │   ├── reuse
│   │   │   ├── appvar1
│   │   │   │   ├── i18n
│   │   │   │   │   ├── i18n_de.properties
│   │   │   │   │   └── i18n_en.properties
│   │   │   │   ├── images
│   │   │   │   │   ├── sports
│   │   │   │   │   │   ├── header_sports.jpg
│   │   │   │   │   │   └── header_sports_small.jpg
│   │   │   │   │   └── travel
│   │   │   │   │       ├── header_travel.jpg
│   │   │   │   │       └── header_travel_small.jpg
│   │   │   │   └── terminologies
│   │   │   │       ├── sports
│   │   │   │       │   ├── i18n.terminologies.sports_de.properties
│   │   │   │       │   └── i18n.terminologies.sports_en.properties
│   │   │   │       └── travel
│   │   │   │           ├── i18n.terminologies.travel_de.properties
│   │   │   │           └── i18n.terminologies.travel_en.properties
│   │   │   └── appvar2
│   │   │       ├── i18n
│   │   │       │   ├── i18n_de.properties
│   │   │       │   └── i18n_en.properties
│   │   │       ├── images
│   │   │       │   └── travel
│   │   │       │       ├── header_travel.jpg
│   │   │       │       └── header_travel_small.jpg
│   │   │       └── terminologies
│   │   │           └── travel
│   │   │               ├── i18n.terminologies.travel_de.properties
│   │   │               └── i18n.terminologies.travel_en.properties
│   │   ├── test
│   │   │   ├── integration
│   │   │   │   ├── AllJourneys.js
│   │   │   │   ├── NavigationJourney.js
│   │   │   │   ├── opaTests.qunit.html
│   │   │   │   ├── opaTests.qunit.js
│   │   │   │   ├── arrangements
│   │   │   │   │   └── Startup.js
│   │   │   │   └── pages
│   │   │   │       ├── App.js
│   │   │   │       ├── Home.js
│   │   │   │       ├── Settings.js
│   │   │   │       └── Statistics.js
│   │   │   └── unit
│   │   │       ├── AllTests.js
│   │   │       ├── model
│   │   │       │   ├── formatter.js
│   │   │       │   └── models.js
│   │   │       ├── unitTests.qunit.html
│   │   │       └── unitTests.qunit.js
│   │   ├── util
│   │   │   └── ServiceUtil.js
│   │   ├── webapp
│   │   │   ├── Component.js
│   │   │   ├── index.html
│   │   │   ├── manifest.json
│   │   │   └── view
│   │   │       ├── App.view.xml
│   │   │       ├── Assistant.view.xml
│   │   │       ├── Files.view.xml
│   │   │       ├── Home.view.xml
│   │   │       ├── Settings.view.xml
│   │   │       └── Sources.view.xml
│   │   ├── ui5-deploy.yaml
│   │   ├── ui5.yaml
│   │   ├── xs-app.json
│   │   ├── README.md
│   │   ├── package-lock.json
│   │   └── package.json
│   ├── annotations.cds
│   └── services.cds
├── db
│   ├── src
│   │   └── .hdiconfig
│   ├── schema.cds
│   └── undeploy.json
├── srv
│   ├── AICoreUtil.js
│   ├── AICoreUtilChatbot.js
│   ├── api.cds
│   ├── api.js
│   ├── AxiosUtil.js
│   ├── bucketUtil.js
│   ├── CFUtil.js
│   ├── DataServiceUtil.js
│   ├── server.js
│   ├── sharepoint.js
│   └── util
│       ├── AxiosUtil.js
│       └── DataServiceUtil.js
├── .eslintrc
├── .gitignore
├── mta.yaml
├── package-lock.json
├── package.json
├── README.md
└── xs-security.json
```

## 2. CAP Application Files Overview

| File Name                      | Description                                                                 |
|--------------------------------|-----------------------------------------------------------------------------|
| `schema.cds`                   | Defines the data model for the CAP application, including entities and types. |
| `undeploy.json`                | Specifies files to be undeployed from the HANA database.                     |
| `.hdiconfig`                   | Configuration for HANA Deployment Infrastructure (HDI) plugins.              |
| `api.cds`                      | Defines the service layer, including entities and functions exposed by the API. |
| `server.js`                    | Main server file to bootstrap the CAP application and handle WebSocket connections. |
| `AICoreUtil.js`                | Utility functions for interacting with AI Core services.                     |
| `AICoreUtilChatbot.js`         | Utility functions for chatbot interactions using AI Core services.           |
| `CFUtil.js`                    | Utility functions for interacting with Cloud Foundry services.               |
| `bucketUtil.js`                | Utility functions for managing AWS S3 buckets and objects.                   |
| `api.js`                       | Implementation of service handlers for the API service defined in `api.cds`. |
| `AxiosUtil.js`                 | Utility class for making HTTP requests with token management.                |
| `DataServiceUtil.js`           | Utility class for data service interactions, including fetching and managing bid cases. |
| `sharepoint.js`                | Utility functions for connecting and interacting with SharePoint services.   |

## 3. Data Model Representation

```plaintext
namespace opportunityassistant;

entity EntityObject : managed, cuid {
    external_id : String(100);
    description : String;
    metadata    : String(5000);
    status      : STATUS default 'INITIAL';
    files       : Association to many EntityObjectFile on files.ID = $self.ID
}

entity EntityObjectFile : cuid {
    filename     : String;
    location     : String;
    size         : String;
    type         : String;
    comments     : String;
    entityObject : Association to one EntityObject;
}

type STATUS : String enum {
    INITIAL                   = 'INITIAL';
    FILES_UPLOADED            = 'FILES_UPLOADED';
    FILES_STORED_IN_VECTOR_DB = 'FILES_STORED_IN_VECTOR_DB';
}
```

## 4. DB Schema

| Name              | Type   | Fields                                                                 | Annotations | Description                                      |
|-------------------|--------|------------------------------------------------------------------------|-------------|--------------------------------------------------|
| `EntityObject`    | Table  | `external_id` (String, key), `description` (String), `metadata` (String), `status` (STATUS, default 'INITIAL'), `files` (Association) | [N/A]       | Represents an object with associated files.      |
| `EntityObjectFile`| Table  | `filename` (String), `location` (String), `size` (String), `type` (String), `comments` (String), `entityObject` (Association) | [N/A]       | Represents a file associated with an entity object. |
| `STATUS`          | Type   | Enum values: `INITIAL`, `FILES_UPLOADED`, `FILES_STORED_IN_VECTOR_DB` | [N/A]       | Represents the status of an entity object.       |

## 5. CDS Definitions

| Name              | Projected DB Entity | Supported CRUD Operations | Fields                                                                 | Description                                      | Annotations |
|-------------------|---------------------|---------------------------|------------------------------------------------------------------------|--------------------------------------------------|-------------|
| `EntityObject`    | `db.EntityObject`   | Create, Read, Update, Delete | `external_id` (String), `description` (String), `metadata` (String), `status` (STATUS), `files` (Association) | Represents an object with associated files.      | [N/A]       |
| `EntityObjectFile`| `db.EntityObjectFile` | Create, Read, Update, Delete | `filename` (String), `location` (String), `size` (String), `type` (String), `comments` (String), `entityObject` (Association) | Represents a file associated with an entity object. | [N/A]       |

## 6. Function and Action Imports

| Name                               | Type      | Description                                      | Supported Operation | Parameters                                     | Return Type | Annotations | Associated Entities |
|------------------------------------|-----------|--------------------------------------------------|---------------------|-----------------------------------------------|-------------|-------------|---------------------|
| `getAdminOptions`                  | Function  | Retrieves admin options.                         | GET                 | [EMPTY]                                       | String      | [N/A]       | [N/A]               |
| `setAdminOptions`                  | Action    | Sets admin options.                              | POST                | `defaultApprover` (String)                    | [EMPTY]     | [N/A]       | [N/A]               |
| `chatMemory`                       | Action    | Performs chat with memory.                       | POST                | `query` (String)                              | String      | [N/A]       | [N/A]               |
| `chatHistoryByUserId`              | Action    | Retrieves chat history by user ID.               | POST                | [EMPTY]                                       | String      | [N/A]       | [N/A]               |
| `isChatbotEnabled`                 | Function  | Checks if chatbot is enabled.                    | GET                 | [EMPTY]                                       | Boolean     | [N/A]       | [N/A]               |
| `getSharepointConnectUrl`          | Function  | Retrieves SharePoint connection URL.             | GET                 | [EMPTY]                                       | String      | [N/A]       | [N/A]               |
| `getAzureToken`                    | Function  | Retrieves Azure token.                           | GET                 | [EMPTY]                                       | String      | [N/A]       | [N/A]               |
| `getBidCases`                      | Function  | Retrieves bid cases.                             | GET                 | `top` (Integer), `skip` (Integer)             | Array of String | [N/A]   | [N/A]               |
| `getBidCasesWithDeliverables`      | Function  | Retrieves bid cases with deliverables.           | GET                 | `top` (Integer), `skip` (Integer), `caseId` (String) | String  | [N/A]       | [N/A]               |
| `getStats`                         | Function  | Retrieves statistics.                            | GET                 | [EMPTY]                                       | String      | [N/A]       | [N/A]               |
| `isAzureTokenValid`                | Function  | Checks if Azure token is valid.                  | GET                 | [EMPTY]                                       | String      | [N/A]       | [N/A]               |
| `getFilesByCaseId`                 | Function  | Retrieves files by case ID.                      | GET                 | `caseId` (String)                             | String      | [N/A]       | [N/A]               |
| `getAllFiles`                      | Function  | Retrieves all files.                             | GET                 | [EMPTY]                                       | String      | [N/A]       | [N/A]               |
| `searchFilesByExternalIdOrFileName`| Function  | Searches files by external ID or file name.      | GET                 | `search` (String)                             | String      | [N/A]       | [N/A]               |
| `deleteFilesByCaseId`              | Action    | Deletes files by case ID.                        | POST                | `caseId` (String)                             | String      | [N/A]       | [N/A]               |
| `refreshChat`                      | Action    | Refreshes chat.                                  | POST                | `model` (String)                              | String      | [N/A]       | [N/A]               |
| `loadBidCaseDeliverablesToVectorDB`| Function  | Loads bid case deliverables to vector DB.        | GET                 | `caseId` (String)                             | String      | [N/A]       | [N/A]               |

## 7. Event Handlers

| Handler Type | Operation | Target                      | Implementation Description                                                                 | Helper Functions Used |
|--------------|-----------|-----------------------------|--------------------------------------------------------------------------------------------|-----------------------|
| `on`         | Read      | `getBidCases`               | Fetches bid cases with pagination support.                                                  | `dataServiceUtil.getBidCases` |
| `on`         | Read      | `getBidCasesWithDeliverables` | Fetches bid cases along with associated deliverables.                                       | `dataServiceUtil.getBidCasesWithDeliverables`, `dataServiceUtil.getFilesByCaseId` |
| `on`         | Custom    | `loadBidCaseDeliverablesToVectorDB` | Loads deliverables to vector database for a specific case.                                  | `dataServiceUtil.loadBidCaseDeliverablesToVectorDB` |
| `on`         | Read      | `getFilesByCaseId`          | Retrieves files associated with a specific case ID.                                         | `dataServiceUtil.getFilesByCaseId` |
| `on`         | Delete    | `deleteFilesByCaseId`       | Deletes files associated with a specific case ID.                                           | `dataServiceUtil.deleteFilesByCaseId` |
| `on`         | Read      | `getAllFiles`               | Retrieves all files available in the system.                                                | `dataServiceUtil.getAllFiles` |
| `on`         | Custom    | `refreshChat`               | Refreshes chat memory for a specific user and model.                                        | `dataServiceUtil.refreshChat` |
| `on`         | Read      | `getStats`                  | Retrieves statistical data about files and bid cases.                                       | `dataServiceUtil.getAllFiles` |
| `on`         | Custom    | `getSharepointConnectUrl`   | Generates a SharePoint connection URL based on the context.                                 | `getSharepointConnectUrl` |
| `on`         | Custom    | `getAzureToken`             | Retrieves Azure token using SharePoint connection details.                                  | `connectSharepoint` |
| `on`         | Custom    | `isAzureTokenValid`         | Checks validity of stored Azure token for the user.                                         | `AxiosUtil.checkTokenExpired` |
| `on`         | Update    | `updatePreference`          | Updates user preferences in the system.                                                     | [EMPTY]               |
| `on`         | Read      | `userInfo`                  | Retrieves user information including admin status.                                          | [EMPTY]               |
| `on`         | Read      | `searchFilesByExternalIdOrFileName` | Searches files based on external ID or file name.                                           | `dataServiceUtil.searchFilesByExternalIdOrFileName` |
| `on`         | Read      | `getAdminOptions`           | Retrieves admin options from the system.                                                    | [EMPTY]               |
| `on`         | Update    | `setAdminOptions`           | Sets admin options in the system.                                                           | [EMPTY]               |
| `on`         | Custom    | `chatMemory`                | Performs chat with memory for a specific user.                                              | `dataServiceUtil.chatMemory` |
| `on`         | Read      | `chatHistoryByUserId`       | Retrieves chat history for a specific user.                                                 | `dataServiceUtil.chatHistoryByUserId` |
| `on`         | Read      | `isChatbotEnabled`          | Checks if chatbot functionality is enabled.                                                 | [EMPTY]               |

## 8. Server Helper Functions

| Function Name          | Location                | Description                                                                 | Parameters                                     | Return Type | Implementation Logic Summary |
|------------------------|-------------------------|-----------------------------------------------------------------------------|------------------------------------------------|-------------|------------------------------|
| `getToken`             | `srv/AICoreUtil.js`     | Retrieves authentication token for AI Core services.                        | [EMPTY]                                        | String      | Makes a GET request to obtain token using client credentials. |
| `chat`                 | `srv/AICoreUtilChatbot.js` | Sends a chat query to AI Core chatbot service.                              | `sQuestion` (String)                           | Object      | Sends POST request with query data to chatbot endpoint. |
| `getStatus`            | `srv/AICoreUtil.js`     | Retrieves status from AI Core deployment.                                   | `token` (String, default: [N/A])               | Object      | Makes a GET request to status endpoint with authorization. |
| `createExecution`      | `srv/AICoreUtil.js`     | Initiates a new execution in AI Core.                                       | [EMPTY]                                        | Object      | Sends POST request to create execution with configuration ID. |
| `predict`              | `srv/AICoreUtil.js`     | Performs prediction using AI Core services.                                 | `req` (Object), `token` (String, default: [N/A]) | Object      | Sends POST request with prediction data to AI Core endpoint. |
| `refreshModelServer`   | `srv/AICoreUtil.js`     | Refreshes model server in AI Core.                                          | [EMPTY]                                        | Object      | Makes a GET request to refresh endpoint with authorization. |
| `getTokenCF`           | `srv/CFUtil.js`         | Retrieves authentication token for Cloud Foundry services.                  | [EMPTY]                                        | String      | Sends POST request to obtain token using destination credentials. |
| `sendEmail`            | `srv/CFUtil.js`         | Sends an email using Microsoft Graph API.                                   | `sEmail` (String), `sContent` (String)         | [EMPTY]     | Sends POST request with email data to Microsoft Graph endpoint. |
| `getMSGraphToken`      | `srv/CFUtil.js`         | Retrieves authentication token for Microsoft Graph API.                     | [EMPTY]                                        | String      | Obtains token from destination configuration. |
| `deleteBucket`         | `srv/bucketUtil.js`     | Deletes an AWS S3 bucket and its contents.                                  | `bucketName` (String)                          | [EMPTY]     | Lists and deletes all objects in the bucket before deleting the bucket itself. |
| `listBuckets`          | `srv/bucketUtil.js`     | Lists all AWS S3 buckets in the account.                                    | [EMPTY]                                        | Array       | Makes a request to list all buckets and returns their names. |
| `getObjectContentByKey`| `srv/bucketUtil.js`     | Retrieves content of an object from AWS S3 by key.                          | `sKey` (String)                                | String      | Makes a request to get object content and returns it in base64 format. |
| `listAllMyObjects`     | `srv/bucketUtil.js`     | Lists all objects in a specific AWS S3 bucket.                              | [EMPTY]                                        | Object      | Lists objects with a specific prefix in the bucket. |
| `listAllObjects`       | `srv/bucketUtil.js`     | Lists all objects in the specified AWS S3 bucket.                           | [EMPTY]                                        | Object      | Makes a request to list all objects in the bucket. |
| `listObjectsWithBase64Content` | `srv/bucketUtil.js` | Lists objects with specific metadata and retrieves their content in base64. | `bucketName` (String), `metadataKey` (String), `metadataValue` (String) | Object | Lists objects and filters them based on metadata before retrieving content. |
| `bucketExists`         | `srv/bucketUtil.js`     | Checks if an AWS S3 bucket exists.                                          | `bucketName` (String)                          | Boolean     | Makes a request to check bucket existence and returns true or false. |
| `listObjects`          | `srv/bucketUtil.js`     | Lists objects in a specified AWS S3 bucket.                                 | `bucketName` (String)                          | Array       | Makes a request to list objects and returns their keys. |
| `deleteObjectsWithMetadata` | `srv/bucketUtil.js` | Deletes objects with specific metadata from AWS S3 bucket.                  | `bucketName` (String), `metadataKey` (String), `metadataValue` (String) | [EMPTY] | Lists and deletes objects based on metadata. |
| `uploadFile`           | `srv/bucketUtil.js`     | Uploads a file to AWS S3 bucket.                                            | `bucketName` (String), `fileName` (String), `fileContent` (String), `metadata` (Object) | String | Uploads file with metadata and returns its location. |
| `downloadFile`         | `srv/bucketUtil.js`     | Downloads a file from AWS S3 bucket.                                        | `bucketName` (String), `fileName` (String)     | String      | Retrieves file content from the bucket. |
| `deleteFile`           | `srv/bucketUtil.js`     | Deletes a file from AWS S3 bucket.                                          | `bucketName` (String), `fileName` (String)     | [EMPTY]     | Deletes specified file from the bucket. |
| `createBucket`         | `srv/bucketUtil.js`     | Creates a new AWS S3 bucket.                                                | `bucketName` (String)                          | Boolean     | Creates a bucket and returns true if successful. |

This documentation provides a comprehensive overview of the SAP CAP project, detailing its file structure, data model, database schema, CDS definitions, function and action imports, event handlers, and server helper functions. Each section is designed to aid developers in understanding and utilizing the components of the CAP application effectively.